In [4]:
import os
import sys

import json
import time
import numpy as np

sys.path.append(".")
from preprocessing.scannet_util import g_label_names, g_raw2scannet
from lib.pc_util import read_ply_xyzrgbnormal
from lib.utils import get_eta
# from lib.config import CONF

In [ ]:
scene_name = 'scene0000_00'
data_folder = os.path.join('./data/sl3d_data/scans', scene_name)
mesh_seg_filename = os.path.join(data_folder, '%s_vh_clean_2.0.010000.segs.json'%(scene_name))

with open(mesh_seg_filename) as jsondata:
        d = json.load(jsondata)
        seg = d['segIndices']
        #print len(seg)
segid_to_pointid = {}
for i in range(len(seg)):
    if seg[i] not in segid_to_pointid:
        segid_to_pointid[seg[i]] = []
    segid_to_pointid[seg[i]].append(i)

print(seg)

In [38]:
# Raw points in XYZRGBA
ply_filename = os.path.join(data_folder, '%s_vh_clean_2.ply' % (scene_name))
points = read_ply_xyzrgbnormal(ply_filename)
print(points[1000])

[ 0.50276446  4.9811115   0.57867444 73.         70.         47.
  0.48054093 -0.7761271  -0.40829787]


In [9]:
 # Instances over-segmented segment IDs: annotation on segments
instance_segids = []
labels = []
annotation_filename = os.path.join(data_folder, '%s.aggregation.json'%(scene_name)) # low-res mesh
# annotation_filename = os.path.join(data_folder, '%s_vh_clean.aggregation.json'%(scene_name)) # high-res mesh
#print annotation_filename
with open(annotation_filename) as jsondata:
    d = json.load(jsondata)
    for x in d['segGroups']:
        instance_segids.append(x['segments'])
        labels.append(x['label'])

In [46]:
# Each instance's points
CLASS_NAMES = g_label_names
RAW2SCANNET = g_raw2scannet
instance_points_list = []
instance_labels_list = []
semantic_labels_list = []
# print(points[segid_to_pointid[instance_segids[0][0]], :])
for i in range(len(instance_segids)):
    segids = instance_segids[i]
    pointids = []
    for segid in segids:
        pointids += segid_to_pointid[segid]
    instance_points = points[np.array(pointids),:]
    instance_points_list.append(instance_points)
    instance_labels_list.append(np.ones((instance_points.shape[0], 1))*i)   
    label = RAW2SCANNET[labels[i]]
    label = CLASS_NAMES.index(label)
    semantic_labels_list.append(np.ones((instance_points.shape[0], 1))*label)

In [71]:
# Refactor data format
scene_points = np.concatenate(instance_points_list, 0)
scene_points = scene_points[:,0:9] # XYZ+RGB+NORMAl
instance_labels = np.concatenate(instance_labels_list, 0) 
semantic_labels = np.concatenate(semantic_labels_list, 0)
data = np.concatenate((scene_points, instance_labels, semantic_labels), 1)
print(scene_points[0])
print(data[:15550,10])

[ 3.8378706e+00  8.0876102e+00  1.8599916e+00  8.5000000e+01
  7.5000000e+01  5.7000000e+01 -1.3799611e-01 -9.9036634e-01
 -1.1475773e-02]
[8. 8. 8. ... 2. 2. 2.]
